# Lorentz-Equivariant Quantum Graph Neural Network (Lorentz-EQGNN)

In [1]:
# For Colab
!pip install torch_geometric
# !pip install torch_sparse
# !pip install torch_scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.0 MB/s eta 0:00:0000:01


In [2]:
!pip install pennylane qiskit pennylane-qiskit pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 70.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12

In [3]:
import pennylane as qml
import qiskit
print(qml.__version__)
print(qiskit.__version__)
import pennylane_qiskit
print(pennylane_qiskit.__version__)
import pennylane as qml
from pennylane import numpy as np
# from pennylane_qiskit import AerDevice

0.38.0
1.2.4
0.38.1


## Dataset

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import h5py
from matplotlib.colors import LogNorm
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
import urllib

# Downloading Dataset
def download_file(url, filename):
    urllib.request.urlretrieve(url, filename)

photon_url = 'https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
electron_url = 'https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'

download_file(photon_url, 'photon.hdf5')
download_file(electron_url, 'electron.hdf5')

def electron_photon(sample=100000):
    file_path_electron = "/kaggle/working/electron.hdf5"
    with h5py.File(file_path_electron, "r") as file:
        X_e = np.array(file["X"])
        y_e = np.array(file["y"])

    file_path_photon = "/kaggle/working/photon.hdf5"
    with h5py.File(file_path_photon, "r") as file:
        X_p = np.array(file["X"])
        y_p = np.array(file["y"])

    X = np.concatenate((X_e[:sample], X_p[:sample]), axis=0)
    y = np.concatenate((y_e[:sample], y_p[:sample]), axis=0)

    return X, y

In [4]:
X, y = electron_photon(sample=100000)
x_red = X
y_red = y
x_red.shape, y_red
jets_photon = x_red[y_red == 1][:10]
jets_photon.shape

(10, 32, 32, 2)

In [5]:
import torch
import numpy as np
import os
import h5py
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

def save_electron_photon_tensors(num_data_per_class=500, max_nodes=139, save_dir="electron_photon/data"):
    """
    Generate and save tensor data files for Electron-Photon dataset in a graph-like format.

    Args:
        num_data_per_class (int): Number of samples to use per class
        max_nodes (int): Maximum number of nodes per graph
        save_dir (str): Directory to save the processed data
    """
    os.makedirs(save_dir, exist_ok=True)

    # Load electron and photon data
    with h5py.File("electron.hdf5", "r") as file:
        X_e = np.array(file["X"])[:num_data_per_class]
        y_e = np.ones(num_data_per_class)

    with h5py.File("photon.hdf5", "r") as file:
        X_p = np.array(file["X"])[:num_data_per_class]
        y_p = np.zeros(num_data_per_class)

    # Combine data
    X = np.concatenate((X_e, X_p), axis=0)
    labels = np.concatenate((y_e, y_p), axis=0)

    # Shuffle the data
    shuffle_idx = np.random.permutation(len(X))
    X = X[shuffle_idx]
    labels = labels[shuffle_idx]

    def image_to_nodes(image_data, max_nodes=139, threshold=1e-4):
        """Convert image data to graph nodes with position and feature information."""
        batch_size, height, width, channels = image_data.shape
        nodes = np.zeros((batch_size, max_nodes, 8))  # 8-dimensional node features
        p4s = np.zeros((batch_size, max_nodes, 4))    # Position and momentum information
        atom_masks = np.zeros((batch_size, max_nodes), dtype=bool)

        for b in range(batch_size):
            # Combine track and ECAL information
            combined = np.sum(image_data[b, :, :, :2], axis=-1)

            # Find significant points and their values
            significant_points = np.where(combined > threshold)
            values = combined[significant_points]

            # Sort points by their values and take exactly max_nodes
            sorted_indices = np.argsort(-values)
            n_points = min(len(sorted_indices), max_nodes)
            selected_indices = sorted_indices[:n_points]

            for idx_pos, idx in enumerate(selected_indices):
                h, w = significant_points[0][idx], significant_points[1][idx]

                # Enhanced node features (8-dimensional)
                nodes[b, idx_pos] = [
                    image_data[b, h, w, 0],  # track energy
                    image_data[b, h, w, 1],  # ECAL energy
                    h / height,              # normalized height position
                    w / width,               # normalized width position
                    np.sqrt(h**2 + w**2) / np.sqrt(height**2 + width**2),  # normalized radius
                    np.arctan2(h - height/2, w - width/2) / np.pi,         # angular position
                    values[idx] / np.max(values),                          # normalized energy
                    1.0 if image_data[b, h, w, 0] > image_data[b, h, w, 1] else 0.0  # track vs ECAL dominance
                ]

                # Create p4s (x, y, E, 0) - normalized coordinates
                x = (w - width/2) / (width/2)   # Center and normalize to [-1, 1]
                y = (h - height/2) / (height/2) # Center and normalize to [-1, 1]
                E = values[idx]                 # Energy value
                p4s[b, idx_pos] = [x, y, E, 0]
                atom_masks[b, idx_pos] = True

        return p4s, nodes, atom_masks

    # Convert image data to graph format
    p4s, nodes, atom_mask = image_to_nodes(X, max_nodes=max_nodes)

    # Convert to torch tensors
    p4s = torch.from_numpy(p4s).float()
    nodes = torch.from_numpy(nodes).float()
    labels = torch.from_numpy(labels).float()
    atom_mask = torch.from_numpy(atom_mask)

    # Create edge mask (fully connected graph between valid nodes)
    edge_mask = atom_mask.unsqueeze(1) * atom_mask.unsqueeze(2)
    diag_mask = ~torch.eye(edge_mask.size(1), dtype=torch.bool).unsqueeze(0)
    edge_mask = edge_mask * diag_mask

    # Calculate edges
    batch_size = len(X)
    rows, cols = [], []
    for batch_idx in range(batch_size):
        nn = batch_idx * max_nodes
        x = coo_matrix(edge_mask[batch_idx])
        rows.append(nn + x.row)
        cols.append(nn + x.col)
    rows = np.concatenate(rows)
    cols = np.concatenate(cols)
    edges = np.stack([rows, cols])

    # Save tensors
    torch.save(p4s, os.path.join(save_dir, "p4s.pt"))
    torch.save(nodes, os.path.join(save_dir, "nodes.pt"))
    torch.save(labels, os.path.join(save_dir, "labels.pt"))
    torch.save(atom_mask, os.path.join(save_dir, "atom_mask.pt"))
    np.save(os.path.join(save_dir, "edge_mask.npy"), edge_mask.numpy())
    np.save(os.path.join(save_dir, "edges.npy"), edges)

    print(f"Saved tensor files to {save_dir}")
    print(f"Shapes:")
    print(f"p4s: {p4s.shape}")
    print(f"nodes: {nodes.shape}")
    print(f"labels: {labels.shape}")
    print(f"atom_mask: {atom_mask.shape}")
    print(f"edge_mask: {edge_mask.shape}")
    print(f"edges: {edges.shape}")

    # Print label distribution
    unique_labels, counts = np.unique(labels, return_counts=True)
    print("\nLabel distribution:")
    for label, count in zip(unique_labels, counts):
        print(f"Class {label}: {count} samples")

if __name__ == '__main__':
    # Generate data for electron (1) vs photon (0)
    save_electron_photon_tensors(
        num_data_per_class=500,  # 500 samples per class = 1000 total
        max_nodes=200,           # Maximum number of nodes per graph
        save_dir="electron_photon/data"
    )

Saved tensor files to electron_photon/data
Shapes:
p4s: torch.Size([1000, 200, 4])
nodes: torch.Size([1000, 200, 8])
labels: torch.Size([1000])
atom_mask: torch.Size([1000, 200])
edge_mask: torch.Size([1000, 200, 200])
edges: (2, 420236)

Label distribution:
Class 0.0: 500 samples
Class 1.0: 500 samples


In [6]:
def get_adj_matrix(n_nodes, batch_size, edge_mask):
    rows, cols = [], []
    for batch_idx in range(batch_size):
        nn = batch_idx*n_nodes
        x = coo_matrix(edge_mask[batch_idx])
        rows.append(nn + x.row)
        cols.append(nn + x.col)
    rows = np.concatenate(rows)
    cols = np.concatenate(cols)

    edges = [torch.LongTensor(rows), torch.LongTensor(cols)]
    return edges

def collate_fn(data):
    data = list(zip(*data)) # label p4s nodes atom_mask
    data = [torch.stack(item) for item in data]
    batch_size, n_nodes, _ = data[1].size()
    atom_mask = data[-1]
    edge_mask = data[-2]

    edges = get_adj_matrix(n_nodes, batch_size, edge_mask)
    return data + [edges]

In [8]:
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import torch

def create_stratified_split(dataset, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, random_state=42):
    """
    Creates stratified train/val/test splits for binary classification.
    """
    # Extract labels and convert to integer labels for stratification
    labels = dataset.tensors[0].numpy()
    labels_int = labels.astype(int)  # Convert to int for stratification

    # Print initial distribution
    unique_classes, class_counts = np.unique(labels_int, return_counts=True)
    print("\nInitial class distribution:")
    for class_idx, count in zip(unique_classes, class_counts):
        print(f"Class {class_idx}: {count} samples ({count/len(labels)*100:.2f}%)")

    # First split: separate train from temporary val+test
    first_sss = StratifiedShuffleSplit(n_splits=1,
                                      train_size=train_ratio,
                                      random_state=random_state)

    train_idx, temp_idx = next(first_sss.split(np.zeros(len(labels)), labels_int))

    # Second split: separate val and test from temporary set
    temp_labels = labels_int[temp_idx]
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)

    second_sss = StratifiedShuffleSplit(n_splits=1,
                                       train_size=val_ratio_adjusted,
                                       random_state=random_state)

    val_idx_temp, test_idx_temp = next(second_sss.split(np.zeros(len(temp_labels)), temp_labels))

    # Convert temporary indices to original dataset indices
    val_idx = temp_idx[val_idx_temp]
    test_idx = temp_idx[test_idx_temp]

    # Create subset datasets
    train_dataset = Subset(dataset, train_idx)
    val_dataset = Subset(dataset, val_idx)
    test_dataset = Subset(dataset, test_idx)

    # Verify class distribution in each split
    def get_split_distribution(indices):
        split_labels = labels_int[indices]
        unique, counts = np.unique(split_labels, return_counts=True)
        return dict(zip(unique, counts / len(indices)))

    print("\nClass distribution in splits:")
    print("Train:", get_split_distribution(train_idx))
    print("Val:", get_split_distribution(val_idx))
    print("Test:", get_split_distribution(test_idx))

    return {
        "train": train_dataset,
        "val": val_dataset,
        "test": test_dataset
    }

# Let's reload the data and ensure we have binary labels
labels = torch.load('/kaggle/working/electron_photon/data/labels.pt')
p4s = torch.load('/kaggle/working/electron_photon/data/p4s.pt')
nodes = torch.load('/kaggle/working/electron_photon/data/nodes.pt')
atom_mask = torch.load('/kaggle/working/electron_photon/data/atom_mask.pt')
edge_mask = torch.from_numpy(np.load('/kaggle/working/electron_photon/data/edge_mask.npy'))

# Print label statistics before creating dataset
print("Label Statistics:")
print("Shape:", labels.shape)
print("Unique values:", torch.unique(labels))
print("Class distribution:", torch.bincount(labels.long()) / len(labels))

# Create the dataset
dataset_all = TensorDataset(labels, p4s, nodes, atom_mask, edge_mask)

# Create stratified splits
datasets = create_stratified_split(dataset_all)

# Create dataloaders
dataloaders = {
    split: DataLoader(
        dataset,
        batch_size=16,
        pin_memory=False,
        collate_fn=collate_fn,
        drop_last=True if (split == 'train') else False,
        num_workers=0,
        shuffle=(split == 'train')
    )
    for split, dataset in datasets.items()
}

Label Statistics:
Shape: torch.Size([1000])
Unique values: tensor([0., 1.])
Class distribution: tensor([0.5000, 0.5000])

Initial class distribution:
Class 0: 500 samples (50.00%)
Class 1: 500 samples (50.00%)

Class distribution in splits:
Train: {0: 0.5, 1: 0.5}
Val: {0: 0.5, 1: 0.5}
Test: {0: 0.5, 1: 0.5}


/tmp/ipykernel_122/2935664756.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load('/kaggle/working/electron_photon/data/labels.pt')
/tmp/ipykernel_122/2

In [9]:
print(p4s.shape) # p4s
print(nodes.shape) # mass
print(atom_mask.shape) # torch.ones
print(edge_mask.shape) # adj_matrix

torch.Size([1000, 200, 4])
torch.Size([1000, 200, 8])
torch.Size([1000, 200])
torch.Size([1000, 200, 200])


In [10]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7a529ffa5b10>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7a51f1b42e60>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7a51f1b42d40>}

In [11]:
import torch
import numpy as np
import h5py
import os
from scipy.sparse import coo_matrix

def save_physics_tensors(num_data=1000, save_dir="random/data"):
    """
    Generate and save tensor data files for electron-photon analysis.

    Args:
        num_data: Number of data points per class
        save_dir: Directory to save the tensor files
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Load electron-photon data
    def load_data(sample=1000):
        with h5py.File("/kaggle/working/electron.hdf5", "r") as file:
            X_e = np.array(file["X"])[:sample]
            y_e = np.ones(sample)

        with h5py.File("/kaggle/working/photon.hdf5", "r") as file:
            X_p = np.array(file["X"])[:sample]
            y_p = np.zeros(sample)

        X = np.concatenate((X_e, X_p), axis=0)
        y = np.concatenate((y_e, y_p), axis=0)
        return X, y

    # Load raw data
    X, labels = load_data(sample=num_data)
    batch_size = len(X)
    n_nodes = 500  # Fixed number of nodes as in quark-gluon data

    # Process image data into graph format
    def image_to_nodes(image_data, max_nodes=100, threshold=1e-4):
        batch_size = len(image_data)
        nodes = np.zeros((batch_size, max_nodes, 1))  # Match quark-gluon node dimension
        p4s = np.zeros((batch_size, max_nodes, 4))
        atom_masks = np.zeros((batch_size, max_nodes), dtype=bool)

        for b in range(batch_size):
            # Combine track and ECAL information
            combined = np.sum(image_data[b, :, :, :2], axis=-1)
            significant_points = np.where(combined > threshold)
            values = combined[significant_points]

            # Sort points by their values and take top max_nodes
            sorted_indices = np.argsort(-values)
            n_points = min(len(sorted_indices), max_nodes)
            selected_indices = sorted_indices[:n_points]

            for idx_pos, idx in enumerate(selected_indices):
                h, w = significant_points[0][idx], significant_points[1][idx]

                # Create node features similar to quark-gluon format
                track_energy = image_data[b, h, w, 0]
                ecal_energy = image_data[b, h, w, 1]
                total_energy = track_energy + ecal_energy

                # Node feature (single dimension as in quark-gluon)
                nodes[b, idx_pos, 0] = np.log(total_energy + 1)

                # Create pseudo-p4s (pt, eta, phi, m) from position and energy
                pt = total_energy
                eta = (h - 16) / 16  # Convert pixel position to pseudorapidity-like value
                phi = (w - 16) / 16 * np.pi  # Convert pixel position to phi-like value
                mass = 0  # Assume massless particles

                p4s[b, idx_pos] = [pt, eta, phi, mass]
                atom_masks[b, idx_pos] = True

        return p4s, nodes, atom_masks

    # Convert image data to graph format
    p4s, nodes, atom_mask = image_to_nodes(X)

    # Convert to torch tensors
    p4s = torch.from_numpy(p4s).float()
    nodes = torch.from_numpy(nodes).float()
    labels = torch.from_numpy(labels).float()
    atom_mask = torch.from_numpy(atom_mask)

    # Create edge mask
    edge_mask = atom_mask.unsqueeze(1) * atom_mask.unsqueeze(2)
    diag_mask = ~torch.eye(edge_mask.size(1), dtype=torch.bool).unsqueeze(0)
    edge_mask = edge_mask * diag_mask

    # Calculate edges
    rows, cols = [], []
    for batch_idx in range(batch_size):
        nn = batch_idx * n_nodes
        x = coo_matrix(edge_mask[batch_idx])
        rows.append(nn + x.row)
        cols.append(nn + x.col)
    rows = np.concatenate(rows)
    cols = np.concatenate(cols)
    edges = np.stack([rows, cols])

    # Save tensors
    torch.save(p4s, os.path.join(save_dir, "p4s.pt"))
    torch.save(nodes, os.path.join(save_dir, "nodes.pt"))
    torch.save(labels, os.path.join(save_dir, "labels.pt"))
    torch.save(atom_mask, os.path.join(save_dir, "atom_mask.pt"))
    np.save(os.path.join(save_dir, "edge_mask.npy"), edge_mask.numpy())
    np.save(os.path.join(save_dir, "edges.npy"), edges)

    print(f"Saved tensor files to {save_dir}")
    print(f"Shapes:")
    print(f"p4s: {p4s.shape}")
    print(f"nodes: {nodes.shape}")
    print(f"labels: {labels.shape}")
    print(f"atom_mask: {atom_mask.shape}")
    print(f"edge_mask: {edge_mask.shape}")
    print(f"edges: {edges.shape}")

# Generate and save the tensor files
save_physics_tensors(num_data=500)  # This will give 1000 total samples (500 each class)

Saved tensor files to random/data
Shapes:
p4s: torch.Size([1000, 100, 4])
nodes: torch.Size([1000, 100, 1])
labels: torch.Size([1000])
atom_mask: torch.Size([1000, 100])
edge_mask: torch.Size([1000, 100, 100])
edges: (2, 420236)


In [12]:
from torch.utils.data import TensorDataset, random_split

def get_adj_matrix(n_nodes, batch_size, edge_mask):
    rows, cols = [], []
    for batch_idx in range(batch_size):
        nn = batch_idx*n_nodes
        x = coo_matrix(edge_mask[batch_idx])
        rows.append(nn + x.row)
        cols.append(nn + x.col)
    rows = np.concatenate(rows)
    cols = np.concatenate(cols)

    edges = [torch.LongTensor(rows), torch.LongTensor(cols)]
    return edges

def collate_fn(data):
    data = list(zip(*data)) # label p4s nodes atom_mask
    data = [torch.stack(item) for item in data]
    batch_size, n_nodes, _ = data[1].size()
    atom_mask = data[-1]
    # edge_mask = atom_mask.unsqueeze(1) * atom_mask.unsqueeze(2)
    # diag_mask = ~torch.eye(edge_mask.size(1), dtype=torch.bool).unsqueeze(0)
    # edge_mask *= diag_mask

    edge_mask = data[-2]

    edges = get_adj_matrix(n_nodes, batch_size, edge_mask)
    return data + [edges]


p4s = torch.load('random/data/p4s.pt')
nodes = torch.load('random/data/nodes.pt')
labels = torch.load('random/data/labels.pt')
atom_mask = torch.load('random/data/atom_mask.pt')
edge_mask = torch.from_numpy(np.load('random/data/edge_mask.npy'))
edges = torch.from_numpy(np.load('random/data/edges.npy'))


# Create a TensorDataset
dataset_all = TensorDataset(labels, p4s, nodes, atom_mask, edge_mask)

# Define the split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate the lengths for each split
total_size = len(dataset_all)
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - train_size - val_size  # Ensure all data is used

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_all, [train_size, val_size, test_size])

# Create a dictionary to hold the datasets
datasets = {
    "train": train_dataset,
    "val": val_dataset,
    "test": test_dataset
}

dataloaders = {split: DataLoader(dataset,
                                 batch_size=16,
                                 # sampler=train_sampler if (split == 'train') else DistributedSampler(dataset, shuffle=False),
                                 pin_memory=False,
                                 # persistent_workers=True,
                                 collate_fn = collate_fn,
                                 drop_last=True if (split == 'train') else False,
                                 num_workers=0)
                    for split, dataset in datasets.items()}

/tmp/ipykernel_122/1146867464.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p4s = torch.load('random/data/p4s.pt')
/tmp/ipykernel_122/1146867464.py:32: FutureWarning:

In [13]:
# Set desired dimensions
batch_size = 1
n_nodes = 3
device = 'cpu'
dtype = torch.float32

# Print initial shapes
print("Initial shapes:")
print("p4s:", p4s.shape)
print("atom_mask:", atom_mask.shape)
print("edge_mask:", edge_mask.shape)
print("nodes:", nodes.shape)

# Select subset of data
p4s = p4s[:batch_size, :n_nodes, :]
atom_mask = atom_mask[:batch_size, :n_nodes]
edge_mask = edge_mask[:batch_size, :n_nodes, :n_nodes]
nodes = nodes[:batch_size, :n_nodes, :]

print("\nAfter selection shapes:")
print("p4s:", p4s.shape)
print("atom_mask:", atom_mask.shape)
print("edge_mask:", edge_mask.shape)
print("nodes:", nodes.shape)

# Reshape tensors
atom_positions = p4s.view(batch_size * n_nodes, -1).to(device, dtype)
atom_mask = atom_mask.view(batch_size * n_nodes, -1).to(device, dtype)
# Don't reshape edge_mask yet
nodes = nodes.view(batch_size * n_nodes, -1).to(device, dtype)

print("\nAfter reshape shapes:")
print("atom_positions:", atom_positions.shape)
print("atom_mask:", atom_mask.shape)
print("edge_mask:", edge_mask.shape)  # original shape
print("nodes:", nodes.shape)

# Recalculate edges for the subset
from scipy.sparse import coo_matrix
rows, cols = [], []
for batch_idx in range(batch_size):
    nn = batch_idx * n_nodes
    # Convert edge_mask to numpy and remove any extra dimensions
    edge_mask_np = edge_mask[batch_idx].cpu().numpy().squeeze()
    x = coo_matrix(edge_mask_np)
    rows.append(nn + x.row)
    cols.append(nn + x.col)

edges = [torch.LongTensor(np.concatenate(rows)).to(device),
         torch.LongTensor(np.concatenate(cols)).to(device)]

# Now reshape edge_mask after edges are calculated
edge_mask = edge_mask.reshape(batch_size * n_nodes * n_nodes, -1).to(device)

print("\nFinal shapes:")
print("atom_positions:", atom_positions.shape)
print("atom_mask:", atom_mask.shape)
print("edge_mask:", edge_mask.shape)
print("nodes:", nodes.shape)
print("edges:", [e.shape for e in edges])

Initial shapes:
p4s: torch.Size([1000, 100, 4])
atom_mask: torch.Size([1000, 100])
edge_mask: torch.Size([1000, 100, 100])
nodes: torch.Size([1000, 100, 1])

After selection shapes:
p4s: torch.Size([1, 3, 4])
atom_mask: torch.Size([1, 3])
edge_mask: torch.Size([1, 3, 3])
nodes: torch.Size([1, 3, 1])

After reshape shapes:
atom_positions: torch.Size([3, 4])
atom_mask: torch.Size([3, 1])
edge_mask: torch.Size([1, 3, 3])
nodes: torch.Size([3, 1])

Final shapes:
atom_positions: torch.Size([3, 4])
atom_mask: torch.Size([3, 1])
edge_mask: torch.Size([9, 1])
nodes: torch.Size([3, 1])
edges: [torch.Size([6]), torch.Size([6])]


In [14]:
print("\nFinal shapes:")
print("atom_positions:", atom_positions.shape)
print("atom_mask:", atom_mask.shape)
print("edge_mask:", edge_mask.shape)
print("nodes:", nodes.shape)
print("edges:", [e.shape for e in edges])


Final shapes:
atom_positions: torch.Size([3, 4])
atom_mask: torch.Size([3, 1])
edge_mask: torch.Size([9, 1])
nodes: torch.Size([3, 1])
edges: [torch.Size([6]), torch.Size([6])]


# 3. LorentzNet

In [15]:
# @title
import torch
from torch import nn
import numpy as np



"""Some auxiliary functions"""

def unsorted_segment_sum(data, segment_ids, num_segments):
    r'''Custom PyTorch op to replicate TensorFlow's `unsorted_segment_sum`.
    Adapted from https://github.com/vgsatorras/egnn.
    '''
    result = data.new_zeros((num_segments, data.size(1)))
    result.index_add_(0, segment_ids, data)
    return result

def unsorted_segment_mean(data, segment_ids, num_segments):
    r'''Custom PyTorch op to replicate TensorFlow's `unsorted_segment_mean`.
    Adapted from https://github.com/vgsatorras/egnn.
    '''
    result = data.new_zeros((num_segments, data.size(1)))
    count = data.new_zeros((num_segments, data.size(1)))
    result.index_add_(0, segment_ids, data)
    count.index_add_(0, segment_ids, torch.ones_like(data))
    return result / count.clamp(min=1)

def normsq4(p):
    r''' Minkowski square norm
         `\|p\|^2 = p[0]^2-p[1]^2-p[2]^2-p[3]^2`
    '''
    psq = torch.pow(p, 2)
    return 2 * psq[..., 0] - psq.sum(dim=-1)

def dotsq4(p,q):
    r''' Minkowski inner product
         `<p,q> = p[0]q[0]-p[1]q[1]-p[2]q[2]-p[3]q[3]`
    '''
    psq = p*q
    return 2 * psq[..., 0] - psq.sum(dim=-1)

def normA_fn(A):
    return lambda p: torch.einsum('...i, ij, ...j->...', p, A, p)

def dotA_fn(A):
    return lambda p, q: torch.einsum('...i, ij, ...j->...', p, A, q)

def psi(p):
    ''' `\psi(p) = Sgn(p) \cdot \log(|p| + 1)`
    '''
    return torch.sign(p) * torch.log(torch.abs(p) + 1)


"""Lorentz Group-Equivariant Block"""

class LGEB(nn.Module):
    def __init__(self, n_input, n_output, n_hidden, n_node_attr=0,
                 dropout = 0., c_weight=1.0, last_layer=False, A=None, include_x=False):
        super(LGEB, self).__init__()
        self.c_weight = c_weight
        self.dimension_reducer = nn.Linear(10, 4) # New linear layer for dimension reduction
        n_edge_attr = 2 if not include_x else 10 # dims for Minkowski norm & inner product
        # With include_X = False, not include_x becomes True, so the value of n_edge_attr is 2.
        print('Input size of phi_e: ', n_input)

        self.include_x = include_x
        self.phi_e = nn.Sequential(
            nn.Linear(n_input, n_hidden, bias=False), # n_input * 2 + n_edge_attr
            nn.BatchNorm1d(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU())

        self.phi_h = nn.Sequential(
            nn.Linear(n_hidden + n_input + n_node_attr, n_hidden),
            nn.BatchNorm1d(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output))

        layer = nn.Linear(n_hidden, 1, bias=False)
        torch.nn.init.xavier_uniform_(layer.weight, gain=0.001)

        self.phi_x = nn.Sequential(
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            layer)

        self.phi_m = nn.Sequential(
            nn.Linear(n_hidden, 1),
            nn.Sigmoid())

        self.last_layer = last_layer
        if last_layer:
            del self.phi_x

        self.A = A
        self.norm_fn = normA_fn(A) if A is not None else normsq4
        self.dot_fn = dotA_fn(A) if A is not None else dotsq4


    def m_model(self, hi, hj, norms, dots):
        out = torch.cat([hi, hj, norms, dots], dim=1)
        # Reduce the dimension of 'out' to 4 using a linear layer
        out = self.dimension_reducer(out)
        out = self.phi_e(out)
        # print("m_model output: ", out.shape)
        w = self.phi_m(out)
        out = out * w
        return out

    def m_model_extended(self, hi, hj, norms, dots, xi, xj):
        out = torch.cat([hi, hj, norms, dots, xi, xj], dim=1)
        out = self.phi_e(out)
        w = self.phi_m(out)
        out = out * w
        return out

    def h_model(self, h, edges, m, node_attr):
        i, j = edges
        agg = unsorted_segment_sum(m, i, num_segments=h.size(0))
        agg = torch.cat([h, agg, node_attr], dim=1)
        out = h + self.phi_h(agg)
        return out

    def x_model(self, x, edges, x_diff, m): # norms
        i, j = edges
        trans = x_diff * self.phi_x(m)
        # print("m: ", m.shape)
        # print("trans: ", trans.shape)
        # From https://github.com/vgsatorras/egnn
        # This is never activated but just in case it explosed it may save the train
        trans = torch.clamp(trans, min=-100, max=100)
        # print("trans: ", trans.shape)
        # print("x.size: ", x.size(0))
        agg = unsorted_segment_mean(trans, i, num_segments=x.size(0))
        x = x + agg * self.c_weight # * norms[i, j], smth like that, or norms
        return x

    def minkowski_feats(self, edges, x):
        i, j = edges
        x_diff = x[i] - x[j]
        norms = self.norm_fn(x_diff).unsqueeze(1)
        dots = self.dot_fn(x[i], x[j]).unsqueeze(1)
        norms, dots = psi(norms), psi(dots)
        return norms, dots, x_diff

    def forward(self, h, x, edges, node_attr=None):
        i, j = edges
        norms, dots, x_diff = self.minkowski_feats(edges, x)

        if self.include_x:
            m = self.m_model_extended(h[i], h[j], norms, dots, x[i], x[j])
        else:
            m = self.m_model(h[i], h[j], norms, dots) # [B*N, hidden]
        if not self.last_layer:
            # print("X: ", x)
            x = self.x_model(x, edges, x_diff, m)
            # print("phi_x(X) = ", x, '\n---\n')

        h = self.h_model(h, edges, m, node_attr)
        return h, x, m

class LorentzNet(nn.Module):
    r''' Implementation of LorentzNet.

    Args:
        - `n_scalar` (int): number of input scalars.
        - `n_hidden` (int): dimension of latent space.
        - `n_class`  (int): number of output classes.
        - `n_layers` (int): number of LGEB layers.
        - `c_weight` (float): weight c in the x_model.
        - `dropout`  (float): dropout rate.
    '''
    def __init__(self, n_scalar, n_hidden, n_class = 2, n_layers = 6, c_weight = 1e-3, dropout = 0., A=None, include_x=False):
        super(LorentzNet, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.embedding = nn.Linear(n_scalar, n_hidden)
        self.LGEBs = nn.ModuleList([LGEB(self.n_hidden, self.n_hidden, self.n_hidden,
                                    n_node_attr=n_scalar, dropout=dropout,
                                    c_weight=c_weight, last_layer=(i==n_layers-1), A=A, include_x=include_x)
                                    for i in range(n_layers)])
        self.graph_dec = nn.Sequential(nn.Linear(self.n_hidden, self.n_hidden),
                                       nn.ReLU(),
                                       nn.Dropout(dropout),
                                       nn.Linear(self.n_hidden, n_class)) # classification

    def forward(self, scalars, x, edges, node_mask, edge_mask, n_nodes):
        h = self.embedding(scalars)

        # print("h before (just the first particle): \n", h[0].cpu().detach().numpy())
        for i in range(self.n_layers):
            h, x, _ = self.LGEBs[i](h, x, edges, node_attr=scalars)
        # print("h after (just the first particle): \n", h[0].cpu().detach().numpy())

        h = h * node_mask
        h = h.view(-1, n_nodes, self.n_hidden)
        h = torch.mean(h, dim=1)
        pred = self.graph_dec(h)

        # print("Final preds: \n", pred.cpu().detach().numpy())
        return pred.squeeze(1)

LGEB(self.n_hidden, self.n_hidden, self.n_hidden,\
                                    n_node_attr=n_scalar, dropout=dropout,\
                                    c_weight=c_weight, last_layer=\(i==n_layers-1), A=A, include_x=include_x)
                                    
We are using n_hidden = 4 and n_layers = 6

n_input=n_hidden, n_output=n_hidden, n_hidden=n_hidden, n_node_attr=n_scalar=8

In [16]:
# @title
import torch
import os, json, random, string
import torch.distributed as dist

def makedir(path):
    try:
        os.makedirs(path)
    except OSError:
        pass

def args_init(args):
    r''' Initialize seed and exp_name.
    '''
    if args.seed is None: # use random seed if not specified
        args.seed = np.random.randint(100)
    if args.exp_name == '': # use random strings if not specified
        args.exp_name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    if (args.local_rank == 0): # master
        print(args)
        makedir(f"{args.logdir}/{args.exp_name}")
        with open(f"{args.logdir}/{args.exp_name}/args.json", 'w') as f:
            json.dump(args.__dict__, f, indent=4)

def sum_reduce(num, device):
    r''' Sum the tensor across the devices.
    '''
    if not torch.is_tensor(num):
        rt = torch.tensor(num).to(device)
    else:
        rt = num.clone()
    dist.all_reduce(rt, op=dist.ReduceOp.SUM)
    return rt

from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

class GradualWarmupScheduler(_LRScheduler):
    """ Gradually warm-up(increasing) learning rate in optimizer.
    Proposed in 'Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour'.
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        multiplier: target learning rate = base lr * multiplier if multiplier > 1.0. if multiplier = 1.0, lr starts from 0 and ends up with the base_lr.
        warmup_epoch: target learning rate is reached at warmup_epoch, gradually
        after_scheduler: after target_epoch, use this scheduler(eg. ReduceLROnPlateau)
    Reference:
        https://github.com/ildoonet/pytorch-gradual-warmup-lr
    """

    def __init__(self, optimizer, multiplier, warmup_epoch, after_scheduler=None):
        self.multiplier = multiplier
        if self.multiplier < 1.:
            raise ValueError('multiplier should be greater thant or equal to 1.')
        self.warmup_epoch = warmup_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super(GradualWarmupScheduler, self).__init__(optimizer)

    @property
    def _warmup_lr(self):
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch + 1) / self.warmup_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * (self.last_epoch + 1) / self.warmup_epoch + 1.) for base_lr in self.base_lrs]

    def get_lr(self):
        if self.last_epoch >= self.warmup_epoch - 1:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_last_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        return self._warmup_lr

    def step_ReduceLROnPlateau(self, metrics, epoch=None):
        self.last_epoch = self.last_epoch + 1 if epoch==None else epoch
        if self.last_epoch >= self.warmup_epoch - 1:
            if not self.finished:
                warmup_lr = [base_lr * self.multiplier for base_lr in self.base_lrs]
                for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
                    param_group['lr'] = lr
                self.finished = True
                return
            if epoch is None:
                self.after_scheduler.step(metrics, None)
            else:
                self.after_scheduler.step(metrics, epoch - self.warmup_epoch)
            return

        for param_group, lr in zip(self.optimizer.param_groups, self._warmup_lr):
            param_group['lr'] = lr

    def step(self, epoch=None, metrics=None):
        if type(self.after_scheduler) != ReduceLROnPlateau:
            if self.finished and self.after_scheduler:
                if epoch is None:
                    self.after_scheduler.step(None)
                else:
                    self.after_scheduler.step(epoch - self.warmup_epoch)
                self.last_epoch = self.after_scheduler.last_epoch + self.warmup_epoch + 1
                self._last_lr = self.after_scheduler.get_last_lr()
            else:
                return super(GradualWarmupScheduler, self).step(epoch)
        else:
            self.step_ReduceLROnPlateau(metrics, epoch)

        self._last_lr = [group['lr'] for group in self.optimizer.param_groups]

    def state_dict(self):
        """Returns the state of the scheduler as a :class:`dict`.

        It contains an entry for every variable in self.__dict__ which
        is not the optimizer.
        """
        result = {key: value for key, value in self.__dict__.items() if key != 'optimizer' or key != "after_scheduler"}
        if self.after_scheduler:
            result.update({"after_scheduler": self.after_scheduler.state_dict()})
        return result

    def load_state_dict(self, state_dict):
        after_scheduler_state = state_dict.pop("after_scheduler", None)
        self.__dict__.update(state_dict)
        if after_scheduler_state:
            self.after_scheduler.load_state_dict(after_scheduler_state)


from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

def buildROC(labels, score, targetEff=[0.3,0.5]):
    r''' ROC curve is a plot of the true positive rate (Sensitivity) in the function of the false positive rate
    (100-Specificity) for different cut-off points of a parameter. Each point on the ROC curve represents a
    sensitivity/specificity pair corresponding to a particular decision threshold. The Area Under the ROC
    curve (AUC) is a measure of how well a parameter can distinguish between two diagnostic groups.
    '''
    if not isinstance(targetEff, list):
        targetEff = [targetEff]
    fpr, tpr, threshold = roc_curve(labels, score)
    idx = [np.argmin(np.abs(tpr - Eff)) for Eff in targetEff]
    eB, eS = fpr[idx], tpr[idx]
    return fpr, tpr, threshold, eB, eS

In [18]:
import os
import torch
from torch import nn, optim
import json, time
# import utils_lorentz
import numpy as np
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

from tqdm import tqdm

def run(model, epoch, loader, partition, N_EPOCHS=None):
    if partition == 'train':
        model.train()
    else:
        model.eval()

    res = {'time':0, 'correct':0, 'loss': 0, 'counter': 0, 'acc': 0,
           'loss_arr':[], 'correct_arr':[],'label':[],'score':[]}

    tik = time.time()
    loader_length = len(loader)

    for i, (label, p4s, nodes, atom_mask, edge_mask, edges) in tqdm(enumerate(loader)):
        if partition == 'train':
            optimizer.zero_grad()

        batch_size, n_nodes, _ = p4s.size()
        atom_positions = p4s.view(batch_size * n_nodes, -1).to(device, dtype)
        atom_mask = atom_mask.view(batch_size * n_nodes, -1).to(device)
        edge_mask = edge_mask.reshape(batch_size * n_nodes * n_nodes, -1).to(device)
        nodes = nodes.view(batch_size * n_nodes, -1).to(device,dtype)
        edges = [a.to(device) for a in edges]
        label = label.to(device, dtype).long()

        pred = model(scalars=nodes, x=atom_positions, edges=edges, node_mask=atom_mask,
                         edge_mask=edge_mask, n_nodes=n_nodes)

        predict = pred.max(1).indices
        correct = torch.sum(predict == label).item()
        # print(pred.shape,label.shape)
        loss = loss_fn(pred, label)

        if partition == 'train':
            loss.backward()
            optimizer.step()
        elif partition == 'test':
            # save labels and probilities for ROC / AUC
            # print("Preds ", pred)
            score = torch.nn.functional.softmax(pred, dim = -1)
            # print("Score test ", score)
            # raise
            res['label'].append(label)
            res['score'].append(score)

        res['time'] = time.time() - tik
        res['correct'] += correct
        res['loss'] += loss.item() * batch_size
        res['counter'] += batch_size
        res['loss_arr'].append(loss.item())
        res['correct_arr'].append(correct)

        # if i != 0 and i % args.log_interval == 0:

    running_loss = sum(res['loss_arr'])/len(res['loss_arr'])
    running_acc = sum(res['correct_arr'])/(len(res['correct_arr'])*batch_size)
    avg_time = res['time']/res['counter'] * batch_size
    tmp_counter = res['counter']
    tmp_loss = res['loss'] / tmp_counter
    tmp_acc = res['correct'] / tmp_counter

    if N_EPOCHS:
        print(">> %s \t Epoch %d/%d \t Batch %d/%d \t Loss %.4f \t Running Acc %.3f \t Total Acc %.3f \t Avg Batch Time %.4f" %
             (partition, epoch + 1, N_EPOCHS, i, loader_length, running_loss, running_acc, tmp_acc, avg_time))
    else:
        print(">> %s \t Loss %.4f \t Running Acc %.3f \t Total Acc %.3f \t Avg Batch Time %.4f" %
             (partition, running_loss, running_acc, tmp_acc, avg_time))

    torch.cuda.empty_cache()
    # ---------- reduce -----------
    if partition == 'test':
        res['label'] = torch.cat(res['label']).unsqueeze(-1)
        res['score'] = torch.cat(res['score'])
        res['score'] = torch.cat((res['label'],res['score']),dim=-1)
    res['counter'] = res['counter']
    res['loss'] = res['loss'] / res['counter']
    res['acc'] = res['correct'] / res['counter']
    return res

def train(model, res, N_EPOCHS, model_path, log_path):
    ### training and validation
    os.makedirs(model_path, exist_ok=True)
    os.makedirs(log_path, exist_ok=True)

    for epoch in range(N_EPOCHS):
        train_res = run(model, epoch, dataloaders['train'], partition='train', N_EPOCHS = N_EPOCHS)
        print("Time: train: %.2f \t Train loss %.4f \t Train acc: %.4f" % (train_res['time'],train_res['loss'],train_res['acc']))
        # if epoch % args.val_interval == 0:

        # if (args.local_rank == 0):
        torch.save(model.state_dict(), os.path.join(model_path, "checkpoint-epoch-{}.pt".format(epoch)) )
        with torch.no_grad():
            val_res = run(model, epoch, dataloaders['val'], partition='val')

        # if (args.local_rank == 0): # only master process save
        res['lr'].append(optimizer.param_groups[0]['lr'])
        res['train_time'].append(train_res['time'])
        res['val_time'].append(val_res['time'])
        res['train_loss'].append(train_res['loss'])
        res['train_acc'].append(train_res['acc'])
        res['val_loss'].append(val_res['loss'])
        res['val_acc'].append(val_res['acc'])
        res['epochs'].append(epoch)

        ## save best model
        if val_res['acc'] > res['best_val']:
            print("New best validation model, saving...")
            torch.save(model.state_dict(), os.path.join(model_path,"best-val-model.pt"))
            res['best_val'] = val_res['acc']
            res['best_epoch'] = epoch

        print("Epoch %d/%d finished." % (epoch, N_EPOCHS))
        print("Train time: %.2f \t Val time %.2f" % (train_res['time'], val_res['time']))
        print("Train loss %.4f \t Train acc: %.4f" % (train_res['loss'], train_res['acc']))
        print("Val loss: %.4f \t Val acc: %.4f" % (val_res['loss'], val_res['acc']))
        print("Best val acc: %.4f at epoch %d." % (res['best_val'],  res['best_epoch']))

        json_object = json.dumps(res, indent=4)
        with open(os.path.join(log_path, "train-result-epoch{}.json".format(epoch)), "w") as outfile:
            outfile.write(json_object)

        ## adjust learning rate
        if (epoch < 31):
            lr_scheduler.step(metrics=val_res['acc'])
        else:
            for g in optimizer.param_groups:
                g['lr'] = g['lr']*0.5


def test(model, res, model_path, log_path):
    ### test on best model
    best_model = torch.load(os.path.join(model_path, "best-val-model.pt"), map_location=device)
    model.load_state_dict(best_model)
    with torch.no_grad():
        test_res = run(model, 0, dataloaders['test'], partition='test')

    print("Final ", test_res['score'])
    pred = test_res['score'].cpu()

    np.save(os.path.join(log_path, "score.npy"), pred)
    fpr, tpr, thres, eB, eS  = buildROC(pred[...,0], pred[...,2])
    auc = roc_auc_score(pred[...,0], pred[...,2])

    metric = {'test_loss': test_res['loss'], 'test_acc': test_res['acc'],
              'test_auc': auc, 'test_1/eB_0.3':1./eB[0],'test_1/eB_0.5':1./eB[1]}
    res.update(metric)
    print("Test: Loss %.4f \t Acc %.4f \t AUC: %.4f \t 1/eB 0.3: %.4f \t 1/eB 0.5: %.4f"\
           % (test_res['loss'], test_res['acc'], auc, 1./eB[0], 1./eB[1]))
    json_object = json.dumps(res, indent=4)
    with open(os.path.join(log_path, "test-result.json"), "w") as outfile:
        outfile.write(json_object)

if __name__ == "__main__":

    N_EPOCHS = 55 # 60

    model_path = "models/LorentzNet/"
    log_path = "logs/LorentzNet/"
    # args_init(args)

    ### set random seed
    torch.manual_seed(42)
    np.random.seed(42)

    ### initialize cuda
    # dist.init_process_group(backend='nccl')
    device = 'cpu' #torch.device("cpu")
    dtype = torch.float32

    ### load data
    # dataloaders = retrieve_dataloaders( batch_size,
    #                                     num_data=100000, # use all data
    #                                     cache_dir="datasets/QMLHEP/quark_gluons/",
    #                                     num_workers=0,
    #                                     use_one_hot=True)

    ### create parallel model
    model = LorentzNet(n_scalar = 1, n_hidden = 4, n_class = 2,\
                       dropout = 0.2, n_layers = 1,\
                       c_weight = 1e-3)

    model = model.to(device)

    ### print model and dataset information
    # if (args.local_rank == 0):
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print("Model Size:", pytorch_total_params)
    for (split, dataloader) in dataloaders.items():
        print(f" {split} samples: {len(dataloader.dataset)}")

    ### optimizer
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)

    ### lr scheduler
    base_scheduler = CosineAnnealingWarmRestarts(optimizer, 4, 2, verbose = False)
    lr_scheduler = GradualWarmupScheduler(optimizer, multiplier=1,\
                                                warmup_epoch=5,\
                                                after_scheduler=base_scheduler) ## warmup

    ### loss function
    loss_fn = nn.CrossEntropyLoss()

    ### initialize logs
    res = {'epochs': [], 'lr' : [],\
           'train_time': [], 'val_time': [],  'train_loss': [], 'val_loss': [],\
           'train_acc': [], 'val_acc': [], 'best_val': 0, 'best_epoch': 0}

    ### training and testing
    print("Training...")
    train(model, res, N_EPOCHS, model_path, log_path)
    test(model, res, model_path, log_path)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Input size of phi_e:  4
Model Size: 199
 train samples: 800
 val samples: 100
 test samples: 100
Training...


50it [00:00, 123.87it/s]


>> train 	 Epoch 1/55 	 Batch 49/50 	 Loss 0.6964 	 Running Acc 0.497 	 Total Acc 0.497 	 Avg Batch Time 0.0081
Time: train: 0.41 	 Train loss 0.6964 	 Train acc: 0.4975


7it [00:00, 264.69it/s]


...

>> train 	 Epoch 55/55 	 Batch 49/50 	 Loss 0.6891 	 Running Acc 0.556 	 Total Acc 0.556 	 Avg Batch Time 0.0081
Time: train: 0.41 	 Train loss 0.6891 	 Train acc: 0.5563


7it [00:00, 230.45it/s]
/tmp/ipykernel_122/861624552.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(os.path.join(model_path, "best-val-model.p

>> val 	 Loss 0.6929 	 Running Acc 2.000 	 Total Acc 0.560 	 Avg Batch Time 0.0013
Epoch 54/55 finished.
Train time: 0.41 	 Val time 0.03
Train loss 0.6891 	 Train acc: 0.5563
Val loss: 0.6921 	 Val acc: 0.5600
Best val acc: 0.5700 at epoch 17.


7it [00:00, 217.48it/s]

>> test 	 Loss 0.6906 	 Running Acc 2.179 	 Total Acc 0.610 	 Avg Batch Time 0.0014
Final  tensor([[1.0000, 0.4909, 0.5091],
        [0.0000, 0.5016, 0.4984],
        [1.0000, 0.4915, 0.5085],
        [0.0000, 0.5023, 0.4977],
        [1.0000, 0.5035, 0.4965],
        [0.0000, 0.4963, 0.5037],
        [0.0000, 0.5004, 0.4996],
        [0.0000, 0.5018, 0.4982],
        [1.0000, 0.5002, 0.4998],
        [0.0000, 0.5012, 0.4988],
        [1.0000, 0.5012, 0.4988],
        [1.0000, 0.4853, 0.5147],
        [0.0000, 0.4988, 0.5012],
        [0.0000, 0.4949, 0.5051],
        [1.0000, 0.4980, 0.5020],
        [1.0000, 0.4976, 0.5024],
        [1.0000, 0.4938, 0.5062],
        [1.0000, 0.4930, 0.5070],
        [1.0000, 0.4989, 0.5011],
        [1.0000, 0.5043, 0.4957],
        [1.0000, 0.4870, 0.5130],
        [0.0000, 0.5018, 0.4982],
        [1.0000, 0.4949, 0.5051],
        [0.0000, 0.5002, 0.4998],
        [1.0000, 0.5017, 0.4983],
        [1.0000, 0.4933, 0.5067],
        [1.0000, 0.5019, 

# 4. Proposed


In [19]:
import torch
import pennylane as qml
import torch.nn.functional as F
from torch import nn
from torch_geometric.utils import to_dense_adj

n_qubits = 4

dev = qml.device('default.qubit', wires=n_qubits)
# dev = qml.device("qiskit.aer", wires=n_qubits)


def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def RY_RX_layer(weights):
    """Applies a layer of parametrized RY and RX rotations."""
    for i, w in enumerate(weights):
        qml.RY(w, wires=i)
        qml.RX(w, wires=i)

def full_entangling_layer(n_qubits):
    """Applies CNOT gates between all pairs of qubits."""
    for i in range(n_qubits):
        for j in range(i+1, n_qubits):
            qml.CNOT(wires=[i, j])

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT.
    """
    # In other words it should apply something like :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(nqubits - 1):
        qml.CRZ(np.pi / 2, wires=[i, i + 1])
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.SWAP(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.SWAP(wires=[i, i + 1])


@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat, q_depth, n_qubits):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    # RY_layer(q_input_features)
    qml.AngleEmbedding(features=q_input_features, wires=range(n_qubits), rotation='Z')

    # Sequence of trainable variational layers
    # for k in range(q_depth):
    #     entangling_layer(n_qubits)
    #     RY_RX_layer(q_weights[k])
    #     # RY_layer(q_weights[k])
    for k in range(q_depth):
        if k % 2 == 0:
            entangling_layer(n_qubits)
            RY_layer(q_weights[k])
        else:
            full_entangling_layer(n_qubits)
            RY_RX_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)


class DressedQuantumNet(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self, n_qubits, q_depth = 1, q_delta=0.001):
        """
        Definition of the *dressed* layout.
        """
        print('n_qubits: ', n_qubits)
        super().__init__()
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))

    def forward(self, input_features):
        """
        Optimized forward pass to reduce runtime.
        """

        # Quantum Embedding (U(X))
        q_in = torch.tanh(input_features) * np.pi / 2.0

        # Preallocate output tensor
        batch_size = q_in.shape[0]
        q_out = torch.zeros(batch_size, self.n_qubits, device=q_in.device)

        # Vectorized execution
        for i, elem in enumerate(q_in):
            q_out_elem = torch.hstack(quantum_net(elem, self.q_params, self.q_depth, self.n_qubits)).float()
            q_out[i] = q_out_elem

        return q_out

In [20]:
# @title
import torch
from torch import nn
import numpy as np
import pennylane as qml

"""
    Lorentz-Equivariant Quantum Block (LEQB).

        - Given the Lie generators found (i.e.: through LieGAN, oracle-preserving latent flow, or some other approach
          that we develop further), once the metric tensor J is found via the equation:

                          L.J + J.(L^T) = 0,

          we just have to specify the metric to make the model symmetry-preserving to the corresponding Lie group.
          In the cells below, we can see how the model preserves symmetries (starting with the default Lorentz group),
          and when we change J to some other metric (Euclidean, for example), Lorentz boosts **break** equivariance, while other
          transformations preserve it (rotations, for the example shown in the cells below)
"""
class LEQB(nn.Module):
    def __init__(self, n_input, n_output, n_hidden, n_node_attr=0,
                 dropout = 0., c_weight=1.0, last_layer=False, A=None, include_x=False):
        super(LEQB, self).__init__()
        self.c_weight = c_weight
        self.dimension_reducer = nn.Linear(10, 4) # New linear layer for dimension reduction
        self.dimension_reducer2 = nn.Linear(9, 4) # New linear layer for dimension reduction for phi_h
        n_edge_attr = 2 if not include_x else 10 # dims for Minkowski norm & inner product
        # With include_X = False, not include_x becomes True, so the value of n_edge_attr is 2. n_input = n_hidden = 4
        print('Input size of phi_e: ', n_input)
        self.include_x = include_x

        """
            phi_e: input size: n_qubits -> output size: n_qubits
            n_hidden has to be equal to n_input,
            but this is just considering that this is a simple working example.
        """
        self.phi_e = DressedQuantumNet(n_input)
#         self.phi_e = nn.Sequential(
#             nn.Linear(n_input, n_hidden, bias=False),  # n_input * 2 + n_edge_attr
#             nn.BatchNorm1d(n_hidden),
#             nn.ReLU(),
#             nn.Linear(n_hidden, n_hidden),
#             nn.ReLU())

        n_hidden = n_input # n_input * 2 + n_edge_attr
        self.phi_h = nn.Sequential(
            nn.Linear(n_hidden + n_input + n_node_attr, n_hidden),
            nn.BatchNorm1d(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output))

#         self.phi_h = DressedQuantumNet(n_hidden)

        layer = nn.Linear(n_hidden, 1, bias=False)
        torch.nn.init.xavier_uniform_(layer.weight, gain=0.001)

        self.phi_x = nn.Sequential(
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            layer)

#         self.phi_x = nn.Sequential(
#             DressedQuantumNet(n_hidden),
#             layer)

#         self.phi_m = nn.Sequential(
#             DressedQuantumNet(n_hidden),
#             nn.Linear(n_hidden, 1),
#             nn.Sigmoid())

        self.phi_m = nn.Sequential(
            nn.Linear(n_hidden, 1),
            nn.Sigmoid())

        # self.phi_e = nn.Sequential(
        #     nn.Linear(n_input * 2 + n_edge_attr, n_hidden, bias=False),
        #     nn.BatchNorm1d(n_hidden),
        #     nn.ReLU(),
        #     nn.Linear(n_hidden, n_hidden),
        #     nn.ReLU())

        self.last_layer = last_layer
        if last_layer:
            del self.phi_x

        self.A = A
        self.norm_fn = normA_fn(A) if A is not None else normsq4
        self.dot_fn = dotA_fn(A) if A is not None else dotsq4

    def m_model(self, hi, hj, norms, dots):
        out = torch.cat([hi, hj, norms, dots], dim=1)
        out = self.dimension_reducer(out) # extra
        # print("Before embedding to |psi> : ", out)
        out = self.phi_e(out).squeeze(0)
        w = self.phi_m(out)
        out = out * w
        return out

    def m_model_extended(self, hi, hj, norms, dots, xi, xj):
        out = torch.cat([hi, hj, norms, dots, xi, xj], dim=1)
        out = self.dimension_reducer(out) # extra
        out = self.phi_e(out).squeeze(0)
        w = self.phi_m(out)
        out = out * w
        return out

    def h_model(self, h, edges, m, node_attr):
        i, j = edges
        agg = unsorted_segment_sum(m, i, num_segments=h.size(0))
        agg = torch.cat([h, agg, node_attr], dim=1)
        #agg = self.dimension_reducer2(agg) # extra for phi_h
        out = h + self.phi_h(agg)
        return out

    def x_model(self, x, edges, x_diff, m):
        i, j = edges
        trans = x_diff * self.phi_x(m)
        # From https://github.com/vgsatorras/egnn
        # This is never activated but just in case it explosed it may save the train
        # From https://github.com/vgsatorras/egnn
        # This is never activated but just in case it explosed it may save the train
        trans = torch.clamp(trans, min=-100, max=100)
        agg = unsorted_segment_mean(trans, i, num_segments=x.size(0))
        x = x + agg * self.c_weight
        return x

    def minkowski_feats(self, edges, x):
        i, j = edges
        x_diff = x[i] - x[j]
        norms = self.norm_fn(x_diff).unsqueeze(1)
        dots = self.dot_fn(x[i], x[j]).unsqueeze(1)
        norms, dots = psi(norms), psi(dots)
        return norms, dots, x_diff

    def forward(self, h, x, edges, node_attr=None):
        i, j = edges
        norms, dots, x_diff = self.minkowski_feats(edges, x)

        if self.include_x:
            m = self.m_model_extended(h[i], h[j], norms, dots, x[i], x[j])
        else:
            m = self.m_model(h[i], h[j], norms, dots) # [B*N, hidden]
        if not self.last_layer:
            x = self.x_model(x, edges, x_diff, m)
        h = self.h_model(h, edges, m, node_attr)
        return h, x, m

class LieEQGNN(nn.Module):
    r''' Implementation of LorentzNet.

    Args:
        - `n_scalar` (int): number of input scalars.
        - `n_hidden` (int): dimension of latent space.
        - `n_class`  (int): number of output classes.
        - `n_layers` (int): number of LEQB layers.
        - `c_weight` (float): weight c in the x_model.
        - `dropout`  (float): dropout rate.
    '''
    def __init__(self, n_scalar, n_hidden, n_class = 2, n_layers = 6, c_weight = 1e-3, dropout = 0., A=None, include_x=False):
        super(LieEQGNN, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.embedding = nn.Linear(n_scalar, n_hidden)
        self.LEQBs = nn.ModuleList([LEQB(self.n_hidden, self.n_hidden, self.n_hidden,
                                    n_node_attr=n_scalar, dropout=dropout,
                                    c_weight=c_weight, last_layer=(i==n_layers-1), A=A, include_x=include_x)
                                    for i in range(n_layers)])
        self.graph_dec = nn.Sequential(nn.Linear(self.n_hidden, self.n_hidden),
                                       nn.ReLU(),
                                       nn.Dropout(dropout),
                                       nn.Linear(self.n_hidden, n_class)) # classification

    def forward(self, scalars, x, edges, node_mask, edge_mask, n_nodes):
        h = self.embedding(scalars)

        # print("h before (just the first particle): \n", h[0].cpu().detach().numpy())
        for i in range(self.n_layers):
            h, x, _ = self.LEQBs[i](h, x, edges, node_attr=scalars)

        # print("h after (just the first particle): \n", h[0].cpu().detach().numpy())

        h = h * node_mask
        h = h.view(-1, n_nodes, self.n_hidden)
        h = torch.mean(h, dim=1)
        pred = self.graph_dec(h)
        return pred.squeeze(1)

In [22]:
import os
import torch
from torch import nn, optim
import json, time
# import utils_lorentz
import numpy as np
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

if __name__ == "__main__":

    N_EPOCHS = 55 # 60

    model_path = "models/LieEQGNN/"
    log_path = "logs/LieEQGNN/"
    # utils_lorentz.args_init(args)

    ### set random seed
    torch.manual_seed(42)
    np.random.seed(42)

    ### initialize cpu
    # dist.init_process_group(backend='nccl')
    device = 'cpu' #torch.device("cuda")
    dtype = torch.float32

    ### load data
    # dataloaders = retrieve_dataloaders( batch_size,
    #                                     num_data=100000, # use all data
    #                                     cache_dir="datasets/QMLHEP/quark_gluons/",
    #                                     num_workers=0,
    #                                     use_one_hot=True)

    model = LieEQGNN(n_scalar = 1, n_hidden = 4, n_class = 2,\
                       dropout = 0.2, n_layers = 1,\
                       c_weight = 1e-3)

    model = model.to(device)

    ### print model and dataset information
    # if (args.local_rank == 0):
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print("Model Size:", pytorch_total_params)
    for (split, dataloader) in dataloaders.items():
        print(f" {split} samples: {len(dataloader.dataset)}")

    ### optimizer
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)

    ### lr scheduler
    base_scheduler = CosineAnnealingWarmRestarts(optimizer, 4, 2, verbose = False)
    lr_scheduler = GradualWarmupScheduler(optimizer, multiplier=1,\
                                                warmup_epoch=5,\
                                                after_scheduler=base_scheduler) ## warmup

    ### loss function
    loss_fn = nn.CrossEntropyLoss()

    ### initialize logs
    res = {'epochs': [], 'lr' : [],\
           'train_time': [], 'val_time': [],  'train_loss': [], 'val_loss': [],\
           'train_acc': [], 'val_acc': [], 'best_val': 0, 'best_epoch': 0}

    ### training and testing
    print("Training...")
    train(model, res, N_EPOCHS, model_path, log_path)
    test(model, res, model_path, log_path)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Input size of phi_e:  4
n_qubits:  4
Model Size: 199
 train samples: 800
 val samples: 100
 test samples: 100
Training...


50it [04:29,  5.40s/it]


>> train 	 Epoch 1/55 	 Batch 49/50 	 Loss 0.7467 	 Running Acc 0.502 	 Total Acc 0.502 	 Avg Batch Time 5.3992
Time: train: 269.96 	 Train loss 0.7467 	 Train acc: 0.5025


7it [00:20,  2.97s/it]


...

>> train 	 Epoch 55/55 	 Batch 49/50 	 Loss 0.6928 	 Running Acc 0.497 	 Total Acc 0.497 	 Avg Batch Time 5.2675
Time: train: 263.38 	 Train loss 0.6928 	 Train acc: 0.4975


7it [00:20,  2.97s/it]
/tmp/ipykernel_122/861624552.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(os.path.join(model_path, "best-val-model.pt

>> val 	 Loss 0.6915 	 Running Acc 1.821 	 Total Acc 0.510 	 Avg Batch Time 0.8318
Epoch 54/55 finished.
Train time: 263.38 	 Val time 20.80
Train loss 0.6928 	 Train acc: 0.4975
Val loss: 0.6927 	 Val acc: 0.5100
Best val acc: 0.5400 at epoch 20.


7it [00:21,  3.09s/it]

>> test 	 Loss 0.6858 	 Running Acc 2.393 	 Total Acc 0.670 	 Avg Batch Time 0.8643
Final  tensor([[1.0000, 0.4797, 0.5203],
        [0.0000, 0.5055, 0.4945],
        [1.0000, 0.4939, 0.5061],
        [0.0000, 0.5174, 0.4826],
        [1.0000, 0.5021, 0.4979],
        [0.0000, 0.5050, 0.4950],
        [0.0000, 0.5051, 0.4949],
        [0.0000, 0.5174, 0.4826],
        [1.0000, 0.4786, 0.5214],
        [0.0000, 0.5150, 0.4850],
        [1.0000, 0.5077, 0.4923],
        [1.0000, 0.4847, 0.5153],
        [0.0000, 0.5096, 0.4904],
        [0.0000, 0.4797, 0.5203],
        [1.0000, 0.5027, 0.4973],
        [1.0000, 0.5009, 0.4991],
        [1.0000, 0.4937, 0.5063],
        [1.0000, 0.4845, 0.5155],
        [1.0000, 0.4988, 0.5012],
        [1.0000, 0.4956, 0.5044],
        [1.0000, 0.4872, 0.5128],
        [0.0000, 0.5051, 0.4949],
        [1.0000, 0.4867, 0.5133],
        [0.0000, 0.5151, 0.4849],
        [1.0000, 0.5123, 0.4877],
        [1.0000, 0.4831, 0.5169],
        [1.0000, 0.5170, 